In [ ]:
import pandas as pd 
import numpy as np
df_old = pd.read_stata("test02.dta")
df=df_old.rename(columns={'d01_it097':'税金等調整前当期利益','d01_it103':'税効果'})
df['税効果/純利益']=df['税効果']/df['税金等調整前当期利益']

In [ ]:
def 求平均(l):
    n = len(l)
    if n == 0:
        return np.nan
    sum = 0
    for i in l:
        sum += float(i)
    return sum/n

dic = {}
for index, row in df.iterrows():
    if dic.get(row['nkcode']) is None:
        dic[row['nkcode']] = []
    if not pd.isna(row['税効果']):
        dic[row['nkcode']].append(row['税効果'])

df['税効果平均'] = 0
for k in dic:
    df.税効果平均[df.nkcode==k] = 求平均(dic[k])
df
#df[['nkcode','税効果','average']]

In [ ]:
df_withoutna=df.dropna(subset=['税効果','price'])
df_withoutna

In [ ]:
df_withoutna['分散_前']=np.square(df_withoutna['税効果']-df_withoutna['税効果平均'])
df_withoutna

In [ ]:
dic2 = {}
for index, row in df_withoutna.iterrows():
    if dic2.get(row['nkcode']) is None:
        dic2[row['nkcode']] = []
    if not pd.isna(row['分散_前']):
        dic2[row['nkcode']].append(row['分散_前'])

def 求標準偏差(l):
    n = len(l)
    if n <= 1:
        return np.nan
    sum = 0
    for i in l:
        sum += float(i)
    return np.sqrt(sum/(n-1))

df_withoutna['分散_后'] = 0
for k in dic2:
    df_withoutna.分散_后[df_withoutna.nkcode==k] = 求標準偏差(dic2[k])
df_withoutna=df_withoutna.dropna(subset=['分散_后'])
df_withoutna

In [ ]:
stock5years=df_withoutna['acc'].isin([201203]) | df_withoutna['acc'].isin([201703])
df_withoutna=df_withoutna[stock5years]
df_withoutna

In [ ]:
dic3 = {}
for index, row in df_withoutna.iterrows():
    if dic3.get(row['nkcode']) is None:
        dic3[row['nkcode']] = []
    if not pd.isna(row['acc']):
        dic3[row['nkcode']].append(row['acc'])

dic3

In [ ]:
from pandas.core.frame import DataFrame

select = df_withoutna.nkcode == 0
for key,values in dic3.items():
   if len(values)==2:
    #  print(key)
     select |= df_withoutna.nkcode == int(key)
     #df_stock = DataFrame(df_withoutna['nkcode']==int(key))

df_withoutna[select]

In [ ]:
df_completed=df_withoutna[['nkcode','nkil','税効果平均','分散_后']]
df_completed=df_completed.drop_duplicates()
df_completed

In [ ]:
from sklearn.cluster import KMeans

iris = df_completed
X_iris=iris[['税効果平均', '分散_后']].values

model = KMeans(n_clusters=5) # k-meansモデル
model.fit(X_iris) # モデルをデータに適合
y_km=model.predict(X_iris) # クラスタを予測

iris['cluster']=y_km
iris.plot.scatter(x='税効果平均', y='分散_后', c='cluster', colormap='viridis');

In [ ]:
df_completed = df_completed[df_completed['分散_后']<30000]
df_completed = df_completed[df_completed['税効果平均']<2000]
df_completed = df_completed[df_completed['税効果平均']>-2000]

df_completed

In [ ]:
iris = df_completed
X_iris=iris[['税効果平均', '分散_后']].values

model = KMeans(n_clusters=5) # k-meansモデル
model.fit(X_iris) # モデルをデータに適合
y_km=model.predict(X_iris) # クラスタを予測

iris['cluster']=y_km
iris.plot.scatter(x='税効果平均', y='分散_后', c='cluster', colormap='viridis');

In [ ]:
df_completed['業界'] = df_completed['nkil']//1000%100
df_completed

In [ ]:
業界=df_completed['業界'].unique()
print(業界)
print(len(業界))

In [ ]:
for i in 業界:
  df_completed_gyoukai = df_completed[df_completed['業界']==i]
  print(df_completed_gyoukai)

In [ ]:
for i in 業界:
  df_completed_gyoukai = df_completed[df_completed['業界']==i]
  df_completed_gyoukai.to_csv(str(i)+'.csv',index=False)

In [ ]:
from sklearn.cluster import KMeans

iris = pd.read_csv('53.csv')
X_iris=iris[['税効果平均', '分散_后']].values

model = KMeans(n_clusters=5) # k-meansモデル
model.fit(X_iris) # モデルをデータに適合
y_km=model.predict(X_iris) # クラスタを予測

iris['cluster']=y_km
iris.plot.scatter(x='税効果平均', y='分散_后', c='cluster', colormap='viridis');
